In [ ]:
#!pip install --user pydbtools==5.6.3

In [ ]:
import pandas as pd  # a module which provides the data structures and functions to store and manipulate tables in dataframes
import pydbtools as pydb  # A module which allows SQL queries to be run on the Analytical Platform from Python, see https://github.com/moj-analytical-services/pydbtools
import boto3  # allows you to directly create, update, and delete AWS resources from Python scripts

# sets parameters to view dataframes for tables easier
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
#Variables to be used in this notebook

#this is the athena database we will be storing our tables in
fcsq_database = "fcsq"

#this is the s3 bucket we will be saving data to
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")

#Last full year to be published - including this publication
annual_year = 2023

#Current publication variables
current_year = 2024
current_quarter = 2

#Next publication variables
next_quarter_year = 2024 
next_quarter = 3

In [ ]:
#imports SDP extract data from S3 bucket into a temporary table
sdp_divorce_data_table = pd.read_csv("s3://alpha-family-data/fcsq_processing/Divorce/sdp_process/sdp_divorce_data_raw_2024q3.csv", low_memory=False)
#Converting selected columns to datetime       
sdp_divorce_data_table['case_recpt_date'] = pd.to_datetime(sdp_divorce_data_table['case_recpt_date'],  errors = 'coerce', format = '%Y-%m-%d %H:%M:%S')
sdp_divorce_data_table['dn_proncd_date'] = pd.to_datetime(sdp_divorce_data_table['dn_proncd_date'],  errors = 'coerce', format = '%Y-%m-%d %H:%M:%S')
sdp_divorce_data_table['da_grntd_date'] = pd.to_datetime(sdp_divorce_data_table['da_grntd_date'],  errors = 'coerce', format = '%Y-%m-%d %H:%M:%S')

pydb.dataframe_to_temp_table(sdp_divorce_data_table, "sdp_divorce_data");


In [ ]:
test = pydb.read_sql_query("SELECT * from __temp__.sdp_divorce_data limit 10")
test

In [ ]:
#record count check in extracted data
#sdp_divorce_data_count = pydb.read_sql_query("SELECT count(*) as count from __temp__.sdp_divorce_data")
#sdp_divorce_data_count